In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt

saved_folder = "saved_data"
num_episodes = 1016

# Load baseline data
Thrpt_baseline = np.load(f'{saved_folder}/Baseline_Thrpt.npy')  # (20, 61)
total_values = Thrpt_baseline.size  # 20 * 61 = 1220

# Initialize arrays to store statistics for each episode
episode_numbers = []
better_counts = []
better_percentages = []
avg_improvements = []

for episode_idx in range(1, num_episodes + 1):
    thrpt_path = f'{saved_folder}/Episode_{episode_idx}_Thrpt.npy'
    
    if os.path.exists(thrpt_path):
        thrpt = np.load(thrpt_path)  # (20, 61)
        
        better_mask = thrpt > Thrpt_baseline
        better_count = np.sum(better_mask)
        percentage_better = (better_count / total_values) * 100
        
        # Calculate average improvement only where episode > baseline
        if better_count > 0:
            avg_improvement = np.mean((thrpt - Thrpt_baseline)[better_mask])
        else:
            avg_improvement = 0
        
        # Store statistics
        episode_numbers.append(episode_idx)
        better_counts.append(better_count)
        better_percentages.append(percentage_better)
        avg_improvements.append(avg_improvement)

        # Print stats for this episode
        print(f"Episode {episode_idx}:")
        print(f"  Better than baseline: {better_count}/{total_values} ({percentage_better:.2f}%)")
        print(f"  Avg improvement: {avg_improvement:.4f} Mbps")
        print("----------------------------------")

# Convert to numpy arrays for easier handling
episode_numbers = np.array(episode_numbers)
better_counts = np.array(better_counts)
better_percentages = np.array(better_percentages)
avg_improvements = np.array(avg_improvements)

# Find the best episode
best_idx = np.argmax(better_counts)
# For cases where multiple episodes have same better_count, find the one with highest avg improvement among them
max_count = better_counts[best_idx]
candidates = np.where(better_counts == max_count)[0]
best_candidate = candidates[np.argmax(avg_improvements[candidates])]

print("\n\n✅ FINAL RESULTS:")
print(f"Best Episode: {episode_numbers[best_candidate]}")
print(f"➡️  Values better than baseline: {better_counts[best_candidate]}/{total_values} ({better_percentages[best_candidate]:.2f}%)")
print(f"📈 Avg improvement (where better): {avg_improvements[best_candidate]:.4f} Mbps")

# Plotting the statistics
plt.figure(figsize=(15, 10))

# Plot 1: Percentage better than baseline
plt.subplot(2, 1, 1)
plt.plot(episode_numbers, better_percentages, 'b-', label='Percentage better')
plt.scatter(episode_numbers[best_candidate], better_percentages[best_candidate], 
            color='red', label=f'Best Episode: {episode_numbers[best_candidate]}')
plt.xlabel('Episode Number')
plt.ylabel('Percentage Better than Baseline (%)')
plt.title('Percentage of Values Better Than Baseline')
plt.grid(True)
plt.legend()

# Plot 2: Average improvement where better
plt.subplot(2, 1, 2)
plt.plot(episode_numbers, avg_improvements, 'g-', label='Average improvement')
plt.scatter(episode_numbers[best_candidate], avg_improvements[best_candidate], 
            color='red', label=f'Best Episode: {episode_numbers[best_candidate]}')
plt.xlabel('Episode Number')
plt.ylabel('Average Improvement (Mbps)')
plt.title('Average Improvement Where Better Than Baseline')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()



In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt

saved_folder = "saved_data"
num_episodes = 1016

# Load baseline data
Thrpt_baseline = np.load(f'{saved_folder}/Baseline_Thrpt.npy')  # (20 users, 61 timesteps)
baseline_mean = np.mean(Thrpt_baseline, axis=1)  # Mean across timesteps for each user

# Initialize arrays to store statistics
episode_numbers = []
mean_comparisons = []  # Will store mean comparison results for each episode

for episode_idx in range(1, num_episodes + 1):
    thrpt_path = f'{saved_folder}/Episode_{episode_idx}_Thrpt.npy'
    
    if os.path.exists(thrpt_path):
        thrpt = np.load(thrpt_path)  # (20, 61)
        episode_mean = np.mean(thrpt, axis=1)  # Mean across timesteps for each user
        
        # Compare means
        better_mask = episode_mean > baseline_mean
        worse_mask = baseline_mean > episode_mean
        equal_mask = np.isclose(episode_mean, baseline_mean)
        
        better_count = np.sum(better_mask)
        worse_count = np.sum(worse_mask)
        equal_count = np.sum(equal_mask)
        
        # Calculate mean differences
        mean_diff = episode_mean - baseline_mean
        avg_improvement = np.mean(mean_diff[better_mask]) if better_count > 0 else 0
        avg_degradation = np.mean(-mean_diff[worse_mask]) if worse_count > 0 else 0
        
        # Store results
        episode_numbers.append(episode_idx)
        mean_comparisons.append({
            'better_users': np.where(better_mask)[0],
            'worse_users': np.where(worse_mask)[0],
            'equal_users': np.where(equal_mask)[0],
            'better_count': better_count,
            'worse_count': worse_count,
            'equal_count': equal_count,
            'mean_diff': mean_diff,
            'avg_improvement': avg_improvement,
            'avg_degradation': avg_degradation,
            'overall_mean_diff': np.mean(mean_diff)
        })
        
        # Print comparison for this episode
        print(f"\nEpisode {episode_idx} Mean Comparison:")
        print(f"Users with higher mean: {better_count}/20")
        print(f"Users with lower mean: {worse_count}/20")
        print(f"Users with equal mean: {equal_count}/20")
        print(f"Overall mean difference: {np.mean(mean_diff):.4f} Mbps")
        
        if better_count > 0:
            print(f"Avg improvement for better users: {avg_improvement:.4f} Mbps")
            print("Better performing users:", np.where(better_mask)[0])
        
        if worse_count > 0:
            print(f"Avg degradation for worse users: {avg_degradation:.4f} Mbps")
            print("Worse performing users:", np.where(worse_mask)[0])

# Convert to numpy arrays
episode_numbers = np.array(episode_numbers)

# Find best episode based on overall mean difference
overall_diffs = [comp['overall_mean_diff'] for comp in mean_comparisons]
best_episode_idx = np.argmax(overall_diffs)
best_episode = episode_numbers[best_episode_idx]

print("\n\n✅ FINAL RESULTS - MEAN COMPARISON:")
print(f"Best Episode: {best_episode}")
print(f"Highest overall mean improvement: {overall_diffs[best_episode_idx]:.4f} Mbps")
print(f"Users improved: {mean_comparisons[best_episode_idx]['better_count']}/20")
print(f"Users degraded: {mean_comparisons[best_episode_idx]['worse_count']}/20")

# Plotting the mean comparison results
plt.figure(figsize=(15, 8))

# Plot 1: Overall mean difference across episodes
plt.subplot(2, 1, 1)
plt.plot(episode_numbers, overall_diffs, 'b-', label='Overall mean difference')
plt.scatter(best_episode, overall_diffs[best_episode_idx], 
            color='red', label=f'Best Episode: {best_episode}')
plt.xlabel('Episode Number')
plt.ylabel('Mean Difference (Mbps)')
plt.title('Overall Mean Throughput Difference (Episode - Baseline)')
plt.grid(True)
plt.legend()

# Plot 2: Number of improved/degraded users
plt.subplot(2, 1, 2)
better_counts = [comp['better_count'] for comp in mean_comparisons]
worse_counts = [comp['worse_count'] for comp in mean_comparisons]
plt.plot(episode_numbers, better_counts, 'g-', label='Improved users')
plt.plot(episode_numbers, worse_counts, 'r-', label='Degraded users')
plt.scatter(best_episode, better_counts[best_episode_idx], color='green')
plt.scatter(best_episode, worse_counts[best_episode_idx], color='red')
plt.xlabel('Episode Number')
plt.ylabel('Number of Users')
plt.title('Number of Users with Improved/Degraded Mean Throughput')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import os

saved_folder = "saved_data"
num_episodes = 1016
best_mean_thrpt = -np.inf
best_episode_index = -1

# Store mean throughput for each episode if needed
mean_thrpts = []

for episode_idx in range(1, num_episodes + 1):
    try:
        episode_tag = f'Episode_{episode_idx}'
        filepath = f'{saved_folder}/{episode_tag}_Thrpt.npy'
        if os.path.exists(filepath):
            thrpt = np.load(filepath)  # Shape (20, 61)
            mean_thrpt_first_10_users = np.mean(thrpt[:10])  # Flatten and mean
            mean_thrpts.append(mean_thrpt_first_10_users)

            if mean_thrpt_first_10_users > best_mean_thrpt:
                best_mean_thrpt = mean_thrpt_first_10_users
                best_episode_index = episode_idx
        else:
            print(f"[Warning] Missing file: {filepath}")

    except Exception as e:
        print(f"[Error] Episode {episode_idx}: {str(e)}")

print(f"\n✅ Best episode index (1-based): {best_episode_index}")
print(f"📈 Best mean throughput (first 10 users): {best_mean_thrpt:.4f} Mbps")


In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt

saved_folder = "saved_data"
num_episodes = 1016

best_mean_thrpt = -np.inf
best_episode_index = -1

# Step 1: Find the best episode
for episode_idx in range(1, num_episodes + 1):
    thrpt_path = f'{saved_folder}/Episode_{episode_idx}_Thrpt.npy'
    if os.path.exists(thrpt_path):
        thrpt = np.load(thrpt_path)  # (20, 61)
        mean_thrpt_first_10 = np.mean(thrpt[:10])
        if mean_thrpt_first_10 > best_mean_thrpt:
            best_mean_thrpt = mean_thrpt_first_10
            best_episode_index = episode_idx

print(f"✅ Best Episode: {best_episode_index} with mean throughput (first 10 users): {best_mean_thrpt:.4f} Mbps")

# Step 2: Load baseline data
Thrpt_baseline = np.load(f'{saved_folder}/Baseline_Thrpt.npy')     # (20, 61)
SINR_baseline = np.load(f'{saved_folder}/Baseline_SINR.npy')       # (20, 61)
Intf_baseline = np.load(f'{saved_folder}/Baseline_Intf.npy')       # (20, 61)

# Step 3: Load best episode data
Thrpt_best = np.load(f'{saved_folder}/Episode_{best_episode_index}_Thrpt.npy')
SINR_best = np.load(f'{saved_folder}/Episode_{best_episode_index}_SINR.npy')
Intf_best = np.load(f'{saved_folder}/Episode_{best_episode_index}_Intf.npy')

# Step 4: Compute mean across timesteps (axis=1) for each user
mean_thrpt_baseline = np.mean(Thrpt_baseline, axis=1)
mean_thrpt_best = np.mean(Thrpt_best, axis=1)

mean_sinr_baseline = np.mean(SINR_baseline, axis=1)
mean_sinr_best = np.mean(SINR_best, axis=1)

mean_intf_baseline = np.mean(Intf_baseline, axis=1)
mean_intf_best = np.mean(Intf_best, axis=1)

users = np.arange(1, 21)

# Step 5: Plot all three metrics
plt.figure(figsize=(18, 5))

# Plot Throughput
plt.subplot(1, 3, 1)
plt.plot(users, mean_thrpt_baseline, label='Baseline', marker='o')
plt.plot(users, mean_thrpt_best, label=f'Episode {best_episode_index}', marker='x')
plt.title('Mean Throughput per User')
plt.xlabel('User Index')
plt.ylabel('Throughput (Mbps)')
plt.legend()
plt.grid(True)

# Plot SINR
plt.subplot(1, 3, 2)
plt.plot(users, mean_sinr_baseline, label='Baseline', marker='o')
plt.plot(users, mean_sinr_best, label=f'Episode {best_episode_index}', marker='x')
plt.title('Mean SINR per User')
plt.xlabel('User Index')
plt.ylabel('SINR (dB)')
plt.legend()
plt.grid(True)

# Plot Interference
plt.subplot(1, 3, 3)
plt.plot(users, mean_intf_baseline, label='Baseline', marker='o')
plt.plot(users, mean_intf_best, label=f'Episode {best_episode_index}', marker='x')
plt.title('Mean Interference per User')
plt.xlabel('User Index')
plt.ylabel('Interference (dB or mW)')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
# Folder name
saved_folder = "saved_data"
# Define the dynamic extra_tag for Episode (change this variable to reflect different episodes)
extra_tag_episode = 'Episode_1016'  # Change this to any other episode (e.g., 'Episode_1020')
# extra_tag_episode = 'Episode_723'

# Define the extra_tag for Baseline
extra_tag_baseline = 'Baseline'

# Load the variables for Baseline
SINR_baseline = np.load(f'{saved_folder}/{extra_tag_baseline}_SINR.npy')
Intf_baseline = np.load(f'{saved_folder}/{extra_tag_baseline}_Intf.npy')
SINR_mW_dict_baseline = np.load(f'{saved_folder}/{extra_tag_baseline}_SINR_mW_dict.npy', allow_pickle=True)
Intf_mW_dict_baseline = np.load(f'{saved_folder}/{extra_tag_baseline}_Intf_mW_dict.npy', allow_pickle=True)
Thrpt_baseline = np.load(f'{saved_folder}/{extra_tag_baseline}_Thrpt.npy')

# Load the variables for Episode_1016 (dynamic based on the extra_tag_episode variable)
SINR_episode = np.load(f'{saved_folder}/{extra_tag_episode}_SINR.npy')
Intf_episode = np.load(f'{saved_folder}/{extra_tag_episode}_Intf.npy')
SINR_mW_dict_episode = np.load(f'{saved_folder}/{extra_tag_episode}_SINR_mW_dict.npy', allow_pickle=True)
Intf_mW_dict_episode = np.load(f'{saved_folder}/{extra_tag_episode}_Intf_mW_dict.npy', allow_pickle=True)
Thrpt_episode = np.load(f'{saved_folder}/{extra_tag_episode}_Thrpt.npy')

# Set up the time steps
timesteps = np.arange(1, 62)  # 61 timesteps

# Create 'graphs' folder if it doesn't exist
graphs_folder = 'graphs'
os.makedirs(graphs_folder, exist_ok=True)

# Plot for each user individually and save them
for user in range(SINR_baseline.shape[0]):
    # Create a figure for each user
    
    # Plot SINR for Baseline and the dynamic Episode (Episode_1016 or others)
    plt.figure(figsize=(10, 6))
    plt.plot(timesteps, SINR_baseline[user], label=f'Baseline - User {user + 1}')
    plt.plot(timesteps, SINR_episode[user], label=f'{extra_tag_episode} - User {user + 1}', linestyle='--')
    plt.xlabel('Timestep')
    plt.ylabel('SINR')
    plt.title(f'User {user + 1} - SINR Comparison')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'{graphs_folder}/User_{user + 1}_SINR_Comparison.png')
    plt.close()

    # Plot Interference for Baseline and dynamic Episode
    plt.figure(figsize=(10, 6))
    plt.plot(timesteps, Intf_baseline[user], label=f'Baseline - User {user + 1}')
    plt.plot(timesteps, Intf_episode[user], label=f'{extra_tag_episode} - User {user + 1}', linestyle='--')
    plt.xlabel('Timestep')
    plt.ylabel('Interference')
    plt.title(f'User {user + 1} - Interference Comparison')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'{graphs_folder}/User_{user + 1}_Interference_Comparison.png')
    plt.close()

    # Plot SINR (mW) for Baseline and dynamic Episode
    plt.figure(figsize=(10, 6))
    plt.plot(timesteps, SINR_mW_dict_baseline[user], label=f'Baseline - User {user + 1}')
    plt.plot(timesteps, SINR_mW_dict_episode[user], label=f'{extra_tag_episode} - User {user + 1}', linestyle='--')
    plt.xlabel('Timestep')
    plt.ylabel('SINR (mW)')
    plt.title(f'User {user + 1} - SINR (mW) Comparison')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'{graphs_folder}/User_{user + 1}_SINR_mW_Comparison.png')
    plt.close()

    # Plot Interference (mW) for Baseline and dynamic Episode
    plt.figure(figsize=(10, 6))
    plt.plot(timesteps, Intf_mW_dict_baseline[user], label=f'Baseline - User {user + 1}')
    plt.plot(timesteps, Intf_mW_dict_episode[user], label=f'{extra_tag_episode} - User {user + 1}', linestyle='--')
    plt.xlabel('Timestep')
    plt.ylabel('Interference (mW)')
    plt.title(f'User {user + 1} - Interference (mW) Comparison')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'{graphs_folder}/User_{user + 1}_Interference_mW_Comparison.png')
    plt.close()

    # Plot Throughput for Baseline and dynamic Episode
    plt.figure(figsize=(10, 6))
    plt.plot(timesteps, Thrpt_baseline[user], label=f'Baseline - User {user + 1}')
    plt.plot(timesteps, Thrpt_episode[user], label=f'{extra_tag_episode} - User {user + 1}', linestyle='--')
    plt.xlabel('Timestep')
    plt.ylabel('Throughput')
    plt.title(f'User {user + 1} - Throughput Comparison')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'{graphs_folder}/User_{user + 1}_Throughput_Comparison.png')
    plt.close()

    print(f'Graphs for User {user + 1} have been saved.')

print("All graphs have been saved in the 'graphs' folder.")


In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt

saved_folder = "saved_data"
num_episodes = 1016

best_mean_thrpt = -np.inf
episode_index = 1016

# Step 2: Load baseline data
Thrpt_baseline = np.load(f'{saved_folder}/Baseline_Thrpt.npy')     # (20, 61)
SINR_baseline = np.load(f'{saved_folder}/Baseline_SINR.npy')       # (20, 61)
Intf_baseline = np.load(f'{saved_folder}/Baseline_Intf.npy')       # (20, 61)

# Step 3: Load best episode data
Thrpt_best = np.load(f'{saved_folder}/Episode_{episode_index}_Thrpt.npy')
SINR_best = np.load(f'{saved_folder}/Episode_{episode_index}_SINR.npy')
Intf_best = np.load(f'{saved_folder}/Episode_{episode_index}_Intf.npy')

# Step 4: Compute mean across timesteps (axis=1) for each user
mean_thrpt_baseline = np.mean(Thrpt_baseline, axis=1)
mean_thrpt_best = np.mean(Thrpt_best, axis=1)

mean_sinr_baseline = np.mean(SINR_baseline, axis=1)
mean_sinr_best = np.mean(SINR_best, axis=1)

mean_intf_baseline = np.mean(Intf_baseline, axis=1)
mean_intf_best = np.mean(Intf_best, axis=1)


print(f"✅ Episode Index: {episode_index} with mean throughput (first 10 users): {np.mean(mean_thrpt_best[:10]):.4f} Mbps")

users = np.arange(1, 21)

# Step 5: Plot all three metrics
plt.figure(figsize=(18, 5))

# Plot Throughput
plt.subplot(1, 3, 1)
plt.plot(users, mean_thrpt_baseline, label='Baseline', marker='o')
plt.plot(users, mean_thrpt_best, label=f'Episode {episode_index}', marker='x')
plt.title('Mean Throughput per User')
plt.xlabel('User Index')
plt.ylabel('Throughput (Mbps)')
plt.legend()
plt.grid(True)

# Plot SINR
plt.subplot(1, 3, 2)
plt.plot(users, mean_sinr_baseline, label='Baseline', marker='o')
plt.plot(users, mean_sinr_best, label=f'Episode {episode_index}', marker='x')
plt.title('Mean SINR per User')
plt.xlabel('User Index')
plt.ylabel('SINR (dB)')
plt.legend()
plt.grid(True)

# Plot Interference
plt.subplot(1, 3, 3)
plt.plot(users, mean_intf_baseline, label='Baseline', marker='o')
plt.plot(users, mean_intf_best, label=f'Episode {episode_index}', marker='x')
plt.title('Mean Interference per User')
plt.xlabel('User Index')
plt.ylabel('Interference (dB or mW)')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()
